In this notebook, we will run and verify the SWOT Assimilated DiScharge (SADS) algorithm against data from the SWOT Pepsi-1 challenge. We start by adding the SADS modules to the load path and importing them. 

In [88]:
@everywhere push!(LOAD_PATH,"../src")
using Gvf
using Kalman
using NCDatasets
using Distributions
using Plots

We load the data from the Po dataset file

In [16]:
ds = Dataset("Po.nc");
xs = NCDatasets.group(ds, "XS_Timeseries");
ri = NCDatasets.group(ds, "River_Info");

In [17]:
qwbm = ri["QWBM"][1];
x = xs["X"][:];
W = xs["W"][:, 1];
H = xs["H"][:, 1];
Q = xs["Q"][:, 1];

We then generate an ensemble of discharge, bed slopes, roughness coefficients

In [65]:
nens = 50;
Qe = rand(Normal(qwbm, 200), nens);
ne = rand(Uniform(0.01, 0.05), nens);
S0 = rand(Normal(0.00019, 0.00005), nens);

and then we can generate an ensemble of depth profiles using the GVF model after we calculate a downstream depth as the boundary condition. We use an assumption of normal flow to estimate that depth along with the observed water surface slope, which is
\begin{equation*}
y = \left( \dfrac{n^2 Q^2}{W^2 S_{f}} \right)^{3/10}
\end{equation*}
for a rectangular cross section

In [66]:
sf = -(H[5] - H[1]) / (x[5] - x[1]);
ye = (ne.^2 .* Qe.^2 ./ (W[5]^2 *sf)).^(3/10);

We also set the ensemble downstream bed elevation as 
$$z_{ds,mod} = h_{ds,obs} - y_{bc} $$

In [67]:
ze = H[5] - ye;

In [68]:
sobs = -mean(diff(H[1:5]) ./ diff(x[1:5]));
Se = zeros(5, nens)
he = zeros(5, nens);
for i in 1:nens
    se = -diff((S0[i] / sobs) * H[1:5]) ./ diff(x[1:5])
    Se[:, i] = [se[1]; se]
    #he[:, i] = gvf(Qe[i], mean(ye), ze[i], S0[i], mean(ne), x[1:5], W[1:5]);
    he[:, i] = gvf(Qe[i], mean(ye), ze[i], Se[:, i], mean(ne), x[1:5], W[1:5]);
 end

We will start with a synthetic experiment, wherein the we will use the first 20 members of the model ensemble as the prior ensemble for the assimilation and one random member as the synthetic observations. Let's create the matrices for the assimilation step.

The state matrix will contain the discharge

In [69]:
A = zeros(1, 20); A[:] = Qe[1:20]; A

1×20 Array{Float64,2}:
 888.493  964.804  637.658  373.327  …  856.682  735.206  862.438  1317.03

As we will be assimilating the water surface slope observations, the model-predicted observation matrix will be derived from the water surface profiles simulated by the GVF model.

In [70]:
hhe = he .+ (ze' .+ S0' .* (x[1:5] - x[1]))[end:-1:1,:];
HA = zeros(size(hhe, 1)-1, 20); HA[:] = diff(hhe, 1)[:, 1:20]

4×20 Array{Float64,2}:
 -0.441521   -0.513846  -0.235566   …  -0.358548   -0.463706   -0.632492
 -0.218216   -0.254969  -0.114988      -0.163449   -0.217836   -0.367887
 -0.127374   -0.149531  -0.0663012     -0.0905373  -0.122964   -0.246631
 -0.0882758  -0.104078  -0.0454801     -0.0604844  -0.0832119  -0.193252

We compile the observation vector along with the observation error matrix (selecting a member not contained in the assimilation ensemble).

In [71]:
d = zeros(size(HA, 1)); d[:] = diff(hhe,1)[:, 45]

4-element Array{Float64,1}:
 -0.48705  
 -0.216228 
 -0.117639 
 -0.0776042

In [72]:
E = rand(Normal(0., 1e-8), length(d), 20);

Now let's estimate the discharge from the slope synthetic observations using the Square Root Ensemble Kalman Filter

In [73]:
Qest = sqrtenkf(A, d, HA, E)

1×1 Array{Float64,2}:
 847.429

We can compare that to the synthetic truth, which is 

In [74]:
Qe[45]

832.681155471703

whereas the prior was

In [75]:
mean(Qe[1:20])

839.268505328464

We now turn to using the real observations from the Pepsi challenge dataset

In [85]:
A = zeros(1, 50); A[:] = Qe;
HA = zeros(4, 50); HA[:] = diff(hhe, 1);
d = zeros(4); d[:] = diff(H[1:5]);
E = rand(Normal(0., 1e-2), length(d), 50);

In [104]:
Qest = sqrtenkf(A, d, HA, E), mean(Qe), mean(Q), mean(d)

([1306.25], 858.8770271189103, 421.6117647058823, -0.22500000000000053)

In [103]:
using LaTeXStrings
scatter(A', mean(HA, 1)', xlab="Discharge", ylab=L"\Delta h", legend=:none)